## Laboratorio 04 - Métodos de Monte Carlo
Integrantes:
- Ricardo Méndez
- Melissa Pérez

Repositorio: https://github.com/MelissaPerez09/Lab04-CC3104

### Task 01
1. ¿Cómo afecta la elección de la estrategia de exploración (exploring starts vs soft policy) a la precisión de la evaluación de políticas en los métodos de Monte Carlo?
    
    a. Considere la posibilidad de comparar el desempeño de las políticas evaluadas con y sin explorar los inicios o con diferentes niveles de exploración en políticas blandas.
    - De manera general, Exploring Starts ofrece evaluaciones más precisas pero necesita un control del entorno, mientras que las soft policies ofrecen una solución práctica y más aplicable, aunque con menor cobertura teórica.
    - Al usar Exploring Starts, se fuerza al agente a comenzar desde cualquier estado y acción al azar. Esto ayuda mucho porque garantiza que explore todo el espacio posible, lo que hace que la evaluación de la política sea más precisa.
    - Una soft policy, como una política ε-greedy, permite al agente elegir la mejor acción la mayoría del tiempo, pero de vez en cuando prueba otras acciones. Mantiene la exploración constante mientras aprende.

2. En el contexto del aprendizaje de Monte Carlo fuera de la póliza, ¿cómo afecta la razón de muestreo de importancia a la convergencia de la evaluación de políticas? Explore cómo la razón de muestreo de importancia afecta la estabilidad y la convergencia.
    - En un aprendizaje off-policy, se aprender sobre una política (objetivo) usando datos generados por otra (la de comportamiento). Para ajustar esto, se utiliza muestreo de importancia, que es una razón que indica qué tanto se debe corregir lo aprendido para que refleje la política objetivo.

3. ¿Cómo puede el uso de una soft policy influir en la eficacia del aprendizaje de políticas óptimas en comparación con las políticas deterministas en los métodos de Monte Carlo? Compare el desempeño y los resultados de aprendizaje de las políticas derivadas de estrategias épsilon-greedy con las derivadas de políticas deterministas.
    - Una soft policy como ε-greedy sigue explorando aunque ya tenga buenas acciones, porque siempre deja un espacio pequeño para probar nuevas cosas. Esto es bueno porque ayuda a descubrir mejores soluciones que quizá no se habían considerado.
    - En cambio, una política determinista elige siempre la mejor acción conocida. Es más rápida en tomar decisiones, pero si esa “mejor acción” no es realmente la óptima global, se puede quedar atascada.

4. ¿Cuáles son los posibles beneficios y desventajas de utilizar métodos de Monte Carlo off-policy en comparación con los on-policy en términos de eficiencia de la muestra, costo computacional y velocidad de aprendizaje?
    - Los principales beneficios a considerar es que los algoritmos off-policy ofrecen una eficiencia de muestra porque puede usar cualquier experiencia recolectada, el costo computacional es más alto por el uso de muestreo de importancia, la velocidad de aprendizaje depende de si las políticas no son muy distintas. Por último, mencionar que su principal desventaja es que es más sensible y difícil de ajustar.
    - Por otro lado, con los algoritmos on-policy se tiene el caso contrario a lo mencionado con anterioridad, la eficiencia de muestra usa únicamente la que genera bajo su propia política, el costo computacional es más bajo porque los cálculos son más directos y la velocidad de aprendizaje es más estable pero depende de la exploración. Su principal desventaja es que es menos eficiente con los datos porque deben ser controlados.

### Task 02

In [1]:
import numpy as np
import random

In [2]:
# definicion del entorno
class InventoryEnvironment:
    def __init__(self):
        self.products = ['product_A', 'product_B']
        self.max_stock = 10 # Pueden cambiar este número si gustan
        self.demand = {'product_A': [0, 1, 2], 'product_B': [0, 1, 2]}
        self.restock_cost = {'product_A': 5, 'product_B': 7}
        self.sell_price = {'product_A': 10, 'product_B': 15}
        self.state = None
    
    def reset(self):
        self.state = {product: random.randint(0, self.max_stock) for product in self.products}
        return self.state
    
    def step(self, action):
        reward = 0
        for product in self.products:
            stock = self.state[product]
            restock = action[product]
            self.state[product] = min(self.max_stock, stock + restock)
            demand = random.choice(self.demand[product])
            sales = min(demand, self.state[product])
            self.state[product] -= sales
            reward += sales * self.sell_price[product] - restock * self.restock_cost[product]
        return self.state, reward
    
# Init el ambiente
env = InventoryEnvironment()

In [3]:
# generar episodios
def generate_episode(policy, days=30, start_state=None, start_action=None):
    episode = []
    
    state = env.reset()
    
    if start_state:
        env.state = start_state.copy()
        state = env.state

    for day in range(days):
        if day == 0 and start_action:
            action = start_action
        else:
            action = policy(state)
        
        next_state, reward = env.step(action)
        episode.append((state.copy(), action.copy(), reward))
        state = next_state

    return episode

In [4]:
# exploring starts
def random_action():
    return {
        'product_A': random.randint(0, env.max_stock),
        'product_B': random.randint(0, env.max_stock)
    }

def exploring_starts(num_episodes=500):
    episodes = []
    for _ in range(num_episodes):
        start_state = {
            'product_A': random.randint(0, env.max_stock),
            'product_B': random.randint(0, env.max_stock)
        }
        start_action = random_action()
        episode = generate_episode(policy=random_policy, start_state=start_state, start_action=start_action)
        episodes.append(episode)
    return episodes

In [5]:
# soft policy
epsilon = 0.1

def random_policy(state):
    return {
        'product_A': random.randint(0, 2),
        'product_B': random.randint(0, 2)
    }

def greedy_policy(Q, state):
    state_key = (state['product_A'], state['product_B'])
    if state_key in Q:
        return max(Q[state_key], key=Q[state_key].get)
    else:
        return random_policy(state)

def epsilon_greedy_policy(Q, state):
    if random.random() < epsilon:
        return random_policy(state)
    else:
        return greedy_policy(Q, state)


### Preguntas
1. ¿Cuál es el valor estimado de mantener diferentes niveles de existencias para cada producto?
2. ¿Cómo afecta el valor epsilon en la política blanda al rendimiento?
3. ¿Cuál es el impacto de utilizar el aprendizaje fuera de la política en comparación con el aprendizaje dentro
de la política?